# Introduction

This notebook analyses a data set on executions in the USA since the reinstatement of the death penalty in 1976. The data set is available on kaggle [here](https://www.kaggle.com/datasets/johnny1994/execution-data-in-us), which is an unmodified copy of the data offered by the original source: [Death Penalty Information Center](https://deathpenaltyinfo.org/executions/executions-overview/executions-in-the-u-s-1608-2002-the-espy-file). This data set contains *all* executions since then, not just a subset. The latest data points at the time of compiling this notebook are from early 2023. Lastly, it might be helpful to know that "[...] capital punishment is, in practice, only applied for aggravated murder [...]" (see [wiki](https://en.wikipedia.org/wiki/Capital_punishment_in_the_United_States)).

-----

Data dictionary:

The columns of the data set are for the most part fairly self-explanatory. Two brief comments on items that might not be immediately obvious:

1. Definition of an "execution volunteer":

    "[...] those individuals who waived at least part of their ordinary appeals or who terminated proceedings that would have entitled them to additional process prior to their execution."

    from the webpage of the  [Death Penalty Information Center](https://deathpenaltyinfo.org/executions/executions-overview/execution-volunteers)


2. "Juvenile" refers to the age of the perpetrator at the point in time of the crime, not the execution.


In [1]:
# importing basic modules for EDA and data cleaning

# from standard library
import os
import sys
import time
import re
import warnings
import pprint
from datetime import datetime
from numbers import Number

# third party modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import missingno as msno
from IPython.display import display

# own scripts
# None


# global settings: modules and jupyter

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

pp = pprint.PrettyPrinter(indent=4)

%matplotlib inline
%config Completer.use_jedi = False

In [2]:
def display_missing_values(a_df):
    """Printing out the absolute and relative number of missing values in a dataframe, sorted in ascending order.
    
    Parameters
    ----------
    a_df : pd.DataFrame
        The dataframe for which missing values are to be displayed.
    
    """
    
    # individual features
    n_nulls = a_df.isnull().sum()
    n_rows = a_df.shape[0]
    table = pd.DataFrame({"Count": n_nulls.values,
                          "Ratio %": np.round((n_nulls.values / n_rows) * 100, 1)},
                         index=n_nulls.index)
    
    # across entire DataFrame
    n_cells_total = np.product(a_df.shape)
    n_missing_total = n_nulls.sum()
    ratio_total_perc = n_missing_total / n_cells_total
    
    print(f"Missing values overall: {n_missing_total} ({ratio_total_perc * 100:.1f} %)\n\n")
    print(table.sort_values("Count"))
    
    return

# High-level overview of the dataset

In [3]:
# reading in dataset, first "visual" check

df = pd.read_csv(r"../input/execution-data-in-us/DPIC Execution Database - U.S. Executions.csv")

df.head(10)

,Execution Volunteer,Number of Victims,Juvenile,First Name,Last Name,Middle Name(s),Suffix,Race,Sex,Region,...,Number of Asian Male Victims,Number of Native American Male Victims,Number of Other Race Male Victims,Number of White Female Victims,Number of Black Female Victims,Number of Latino Female Victims,Number of Asian Female Victims,Number of American Indian or Alaska Native Female Victims,Number of Other Race Female Victims,Victim(s) Race(s)
0,yes,1,no,Gary,Gilmore,Mark,NaN,White,Male,West,...,0,0,0,0,0,0,0,0,0,White
1,no,1,no,John,Spenkelink,NaN,NaN,White,Male,South,...,0,0,0,0,0,0,0,0,0,White
2,yes,1,no,Jesse,Bishop,NaN,NaN,White,Male,West,...,0,0,0,0,0,0,0,0,0,White
3,yes,4,no,Steven,Judy,NaN,NaN,White,Male,Midwest,...,0,0,0,2,0,0,0,0,0,White
4,yes,1,no,Frank,Coppola,NaN,NaN,White,Male,South,...,0,0,0,0,0,0,0,0,0,White
5,no,1,no,Charlie,Brooks,NaN,NaN,Black,Male,South,...,0,0,0,0,0,0,0,0,0,White
6,no,1,no,John,Evans,NaN,NaN,White,Male,South,...,0,0,0,0,0,0,0,0,0,White
7,no,1,no,Jimmy,Gray,Lee,NaN,White,Male,South,...,0,0,0,1,0,0,0,0,0,White
8,no,1,no,Robert,Sullivan,NaN,NaN,White,Male,South,...,0,0,0,0,0,0,0,0,0,White
9,no,1,no,Robert,Williams,Wayne,NaN,Black,Male,South,...,0,0,0,0,0,0,0,0,0,Black


In [4]:
# checking for duplicates

n_dupl = df.shape[0] - df.drop_duplicates().shape[0]

print(f"Number of duplicate rows: {n_dupl}")

Number of duplicate rows: 0


In [5]:
# number of columns and rows, and ratio of rows to columns

print(f"Number of rows: {df.shape[0]}")
print(f"Number of cols: {df.shape[1]}\n")

print(f"Initial ratio of rows to features: {df.shape[0] / (df.shape[1] - 1):.0f}")

Number of rows: 1561
Number of cols: 27

Initial ratio of rows to features: 60


In [6]:
# missing values

display_missing_values(df)

Missing values overall: 2617 (6.2 %)


                                                    Count  Ratio %
Execution Volunteer                                     0      0.0
Number of American Indian or Alaska Native Fema...      0      0.0
Number of Asian Female Victims                          0      0.0
Number of Latino Female Victims                         0      0.0
Number of Black Female Victims                          0      0.0
Number of White Female Victims                          0      0.0
Number of Other Race Male Victims                       0      0.0
Number of Native American Male Victims                  0      0.0
Number of Asian Male Victims                            0      0.0
Number of Latino Male Victims                           0      0.0
Number of Black Male Victims                            0      0.0
Number of White Male Victims                            0      0.0
Number of Other Race Female Victims                     0      0.0
Execution Date         

# Data cleaning

**Fixing column names**

In [7]:
# automatic treatment of column names

df.columns = [col_name.strip().lower().replace(" ", "_") for col_name in df.columns]

# sanity check
pp.pprint(list(df.columns))

[   'execution_volunteer',
    'number_of_victims',
    'juvenile',
    'first_name',
    'last_name',
    'middle_name(s)',
    'suffix',
    'race',
    'sex',
    'region',
    'country',
    'state',
    'foreign_national',
    'execution_date',
    'number_of_white_male_victims',
    'number_of_black_male_victims',
    'number_of_latino_male_victims',
    'number_of_asian_male_victims',
    'number_of_native_american_male_victims',
    'number_of_other_race_male_victims',
    'number_of_white_female_victims',
    'number_of_black_female_victims',
    'number_of_latino_female_victims',
    'number_of_asian_female_victims',
    'number_of_american_indian_or_alaska_native_female_victims',
    'number_of_other_race_female_victims',
    'victim(s)_race(s)']


In [8]:
# manual renaming of column names, where needed

cols_to_rename = {
    'execution_volunteer': 'volunteer',
    'number_of_victims': "n_victims",
    'number_of_white_male_victims': 'n_white_male_victims',
    'number_of_black_male_victims': 'n_black_male_victims',
    'number_of_latino_male_victims': 'n_latino_male_victims',
    'number_of_asian_male_victims': 'n_asian_male_victims',
    'number_of_native_american_male_victims': 'n_native_american_male_victims',
    'number_of_other_race_male_victims': 'n_other_race_male_victims',
    'number_of_white_female_victims': 'n_white_female_victims',
    'number_of_black_female_victims': 'n_black_female_victims',
    'number_of_latino_female_victims': 'n_latino_female_victims',
    'number_of_asian_female_victims': 'n_asian_female_victims',
    'number_of_american_indian_or_alaska_native_female_victims': 'n_native_american_female_victims',
    'number_of_other_race_female_victims': 'n_other_race_female_victims', 
}

df = df.rename(columns=cols_to_rename)

# sanity check
pp.pprint(list(df.columns))

[   'volunteer',
    'n_victims',
    'juvenile',
    'first_name',
    'last_name',
    'middle_name(s)',
    'suffix',
    'race',
    'sex',
    'region',
    'country',
    'state',
    'foreign_national',
    'execution_date',
    'n_white_male_victims',
    'n_black_male_victims',
    'n_latino_male_victims',
    'n_asian_male_victims',
    'n_native_american_male_victims',
    'n_other_race_male_victims',
    'n_white_female_victims',
    'n_black_female_victims',
    'n_latino_female_victims',
    'n_asian_female_victims',
    'n_native_american_female_victims',
    'n_other_race_female_victims',
    'victim(s)_race(s)']


**Fixing data types**

In [9]:
# data type conversion to datetime

cols_to_dt = "execution_date"

df[cols_to_dt] = pd.to_datetime(df[cols_to_dt])

**Dropping columns**

In [10]:
# dropping features which are not relevant for analysis
df = df.drop(columns=["first_name", "last_name", "middle_name(s)", "suffix", "country"])

**Fixes for categorical columns**

In [11]:
# automatic treatment of classes
#   remove leading + trailing white spaces
#   converting to lower case

cat_cols = df.select_dtypes(include=["object"]).columns
df[cat_cols] = df[cat_cols].apply(lambda item: item.str.strip().str.lower())

In [12]:
df["race"] = df.race.apply(lambda x: x.replace("american indian or alaska native", "native american"))

In [13]:
df["race"] = df.race.apply(lambda x: x.replace("other race", "other"))

In [14]:
replace_vals = {"multiple (including white)": "multiple", 
                "other race": "other",
                "american indian or alaska native": "native american"}

df = df.replace(replace_vals)

**Fixes for datetime column**

In [15]:
# only year is of interest here

df["execution_date"] = df.execution_date.dt.year

df = df.rename(columns={"execution_date": "year"})

# Analysis of the data

## Executions across time

In [16]:
fig = px.histogram(df, x="year", hover_name="year", labels={"year": "Year"})
fig.update_traces(marker_line_width=1, marker_line_color="white")
fig.update_layout(yaxis_title="Count")
fig.show()

Executions clearly ramped up since their reinstatement in 1976, peaked around the turn of the millenium, and then steadily declined again thereafter.

## Ratio of male to female averaged across the entire time frame

In [17]:
df_temp = ( df.groupby(["sex"])[["state"]].count()
              .rename(columns={"state": "total"})
              .reset_index()
          )

df_temp["sex"] = df_temp.sex.str.title()

df_temp.head()

,sex,total
0,Female,18
1,Male,1543


In [18]:
fig = px.pie(df_temp, values='total', names="sex", hole=0.6, title="Was the executed male or female?")
fig.show()

The executed were almost exclusively male.

## Ratio of juvenile to non-juvenile averaged across the entire time frame

N.B.: This refers to the point in time of the murder(s), not the execution.

In [19]:
df_temp = ( df.groupby(["juvenile"])[["state"]].count()
              .rename(columns={"state": "total"})
              .reset_index()
          )

df_temp["juvenile"] = df_temp.juvenile.str.title()

df_temp.head()

,juvenile,total
0,No,1539
1,Yes,22


In [20]:
fig = px.pie(df_temp, values='total', names="juvenile", hole=0.6, 
             title="Was the executed juvenile at the point in time of the murder(s)?")
fig.show()

The executed were almost exclusively not juvenile when commiting the murder(s).

## Ratio of the different races averaged across the entire time frame

In [21]:
df_temp = ( df.groupby(["race"])[["state"]].count()
              .rename(columns={"state": "total"})
              .reset_index()
          )

df_temp["race"] = df_temp.race.str.title()

df_temp.head()

,race,total
0,Asian,7
1,Black,532
2,Latinx,130
3,Native American,20
4,Other,3


In [22]:
fig = px.pie(df_temp, values='total', names="race", hole=0.6, 
             title="What was the race of the executed?")
fig.show()

Whites and Blacks constitute the clear majority of the executed.

## Ratio of foreign to non-foreign nationals averaged across the entire time frame

In [23]:
df_temp = ( df.groupby(["foreign_national"])[["state"]].count()
              .rename(columns={"state": "total"})
              .reset_index()
          )

df_temp["foreign_national"] = df_temp.foreign_national.str.title()

df_temp.head()

,foreign_national,total
0,No,1527
1,Yes,34


In [24]:
fig = px.pie(df_temp, values='total', names="foreign_national", hole=0.6, title="Was the executed a foreign national?")
fig.show()

The executed were almost exclusively not foreign nationals.

## Ratio of the various US regions averaged across the entire time frame

N.B.: A common way of subdividing the US in "regions" is the following: Northeast, Midwest, West, South

In [25]:
df_temp = ( df.groupby(["region"])[["state"]].count()
              .rename(columns={"state": "total"})
              .reset_index()
          )

df_temp["region"] = df_temp.region.str.title()

df_temp.head()

,region,total
0,Midwest,195
1,Northeast,4
2,South,1273
3,West,89


In [26]:
fig = px.pie(df_temp, values='total', names="region", hole=0.6, 
             title="What was the US region where the execution took place?")
fig.show()

The vast majority of executions were carried out in the South.

## Ratio of the various US states averaged across the entire time frame

In [27]:
df_temp = ( df.groupby(["state"])[["region"]]
              .count()
              .rename(columns={"region": "total"})
              .reset_index()
              .sort_values(by="total", ascending=False)
              .reset_index(drop=True)
          )

df_temp

,state,total
0,texas,579
1,oklahoma,120
2,virginia,113
3,florida,99
4,missouri,94
5,georgia,76
6,alabama,70
7,ohio,56
8,south carolina,43
9,north carolina,43


There is a long "tail" of relatively low-count states. For visualisation purposes only the 12 highest-count states will be shown explicity, the remaining ones will be pooled together under "Other States". 

In [28]:
n_top = 12

df_temp_top = df_temp.iloc[:n_top]

df_temp_top

,state,total
0,texas,579
1,oklahoma,120
2,virginia,113
3,florida,99
4,missouri,94
5,georgia,76
6,alabama,70
7,ohio,56
8,south carolina,43
9,north carolina,43


In [29]:
df_temp_rest = df_temp.iloc[n_top:]

df_temp_rest

,state,total
12,louisiana,28
13,mississippi,23
14,indiana,20
15,federal,16
16,delaware,16
17,tennessee,13
18,california,13
19,nevada,12
20,illinois,12
21,utah,7


In [30]:
sum_rest = df_temp_rest["total"].sum()

sum_rest

197

In [31]:
df_temp2 = pd.DataFrame({"state": ["other states"], "total": [sum_rest]})

df_temp = pd.concat([df_temp_top, df_temp2], axis=0, ignore_index=True)

df_temp

,state,total
0,texas,579
1,oklahoma,120
2,virginia,113
3,florida,99
4,missouri,94
5,georgia,76
6,alabama,70
7,ohio,56
8,south carolina,43
9,north carolina,43


In [32]:
df_temp["state"] = df_temp["state"].str.title()

In [33]:
fig = px.pie(df_temp, names="state", values="total", category_orders={"state": list(df_temp.state)}, 
             title="What was the state where the execution was carried out?")

layout = go.Layout(
    annotations=[
        dict(text='Legend Title'
        )
    ]
)

fig.show()

A substantial part of total number of executions were carried out in Texas.

## Total number of executions since 1976 across the US states

Obtaining a mapping of state name to its abbreviation first as the choropleth map expects the latter.

In [34]:
df_stateabbrev = pd.read_excel("../input/usa-state-abbreviations/state-abbrev.xlsx", header=None)

In [35]:
mapping = {}

for full, abbrev in zip(df_stateabbrev.iloc[:,0], df_stateabbrev.iloc[:,1]):
    mapping[full] = abbrev

# adding the mapping for "Federal"
mapping["Federal"] = "??"

In [36]:
# adding column with state abbreviation to original dataframe

df["state_abbrev"] = df.state.apply(lambda s: mapping[s.title()])

In [37]:
df_temp = ( df.groupby(["state", "state_abbrev"])[["region"]]
              .count()
              .rename(columns={"region": "total"})
              .reset_index()
          )

df_temp["state"] = df_temp.state.str.title()

df_temp.head()

,state,state_abbrev,total
0,Alabama,AL,70
1,Arizona,AZ,40
2,Arkansas,AR,31
3,California,CA,13
4,Colorado,CO,1


In [38]:
fig = px.choropleth(locations=df_temp.state_abbrev, locationmode="USA-states", color=df_temp.total, scope="usa", 
                    hover_name=df_temp.state, color_continuous_scale="solar", title="Where were the executions carried out?")

fig.update_layout(coloraxis_colorbar=dict(
    title="Executions"))

# TODO: format hovertemplate properly -- for now I can't make it take on variables
# fig.update_traces(hovertemplate="<b>%{location}<b>" + "<br><br>" + "%{colors} execution(s)")

fig.show()

As already discussed in the context of the preceding graph, Texas noticably pops out. Futhermore, the relatively low count in the Northeast as well as the West is noticible.

## Histogram of the number of victims for each case averaged across the entire time frame

In [39]:
df_temp = ( df.groupby(["n_victims"])[["state"]]
              .count()
              .rename(columns={"state": "total"})
              .reset_index()
          )

df_temp.head()

,n_victims,total
0,1,1158
1,2,246
2,3,98
3,4,36
4,5,11


In [40]:
fig = px.histogram(df_temp, x="n_victims", y="total", nbins=16, 
                   title="How high was the number of murder victims in each case?", 
                   labels={"n_victims": "Number of Victims", "sum of total": "aaa"})
fig.update_traces(marker_line_width=1, marker_line_color="white")
fig.update_layout(
   xaxis = dict(
      tickmode = 'linear',
      tick0 = 0,
      dtick = 1
   )
)
fig.update_layout(yaxis_title="Count")
fig.show()

In the vast majority of cases, there is a single murder victim. Higher victim numbers exist, however, at much lower frequencies. 

## Ratio of volunteers to non-volunteers averaged across the entire time frame

Definition of an "execution volunteer":

"[...] those individuals who waived at least part of their ordinary appeals or who terminated proceedings that would have entitled them to additional process prior to their execution."

from the webpage of the  [Death Penalty Information Center](https://deathpenaltyinfo.org/executions/executions-overview/execution-volunteers)

In [41]:
df_temp = ( df.groupby(["volunteer"])[["state"]]
              .count()
              .rename(columns={"state": "total"})
              .reset_index()
          )

df_temp["volunteer"] = df_temp.volunteer.str.title()

df_temp.head()

,volunteer,total
0,No,1411
1,Yes,150


In [42]:
fig = px.pie(df_temp, values='total', names="volunteer", hole=0.6, title="What is the proportion of execution volunteers?")
fig.show()

Almost 10% of executions involved a "voluntary execution"!

## Ratio of the different races for the case of a voluntary execution

In [43]:
df_temp = ( df.query("volunteer == 'yes'")
              .groupby(["race"])[["state"]]
              .count()
              .rename(columns={"state": "total"})
              .reset_index()
          )

df_temp

,race,total
0,asian,1
1,black,7
2,latinx,12
3,native american,2
4,white,128


In [44]:
df_temp["race"] = df_temp.race.str.title()

In [45]:
fig = px.pie(df_temp, values='total', names='race', title="What is the racial breakdown of those who volunteered?")
fig.show()

The vast majority of execution volunteers were White.

## Ratio of intra- and inter-racial averaged across the entire time frame

In [46]:
df_temp = df["race"] == df["victim(s)_race(s)"]

df_temp.head()

0    True
1    True
2    True
3    True
4    True
dtype: bool

In [47]:
match_count = df_temp.sum()

df_temp = pd.DataFrame({"matching": ["Yes", "No"], "total": [match_count, df_temp.count()-match_count]})

In [48]:
fig = px.pie(df_temp, values='total', names='matching', title="Do the races of perpetrator and victim(s) coincide?")
fig.show()

In two thirds of all cases the races of perpetrator and victim(s) coincide.

## Ratio of intra- and inter-racial murders averaged across the entire time frame: More granularity

In [49]:
df_temp = ( df.groupby(["race", "victim(s)_race(s)"])[["state"]]
              .count()
              .rename(columns={"state": "total"})
              .reset_index()
          )

df_temp.head()

,race,victim(s)_race(s),total
0,asian,asian,4
1,asian,white,3
2,black,asian,12
3,black,black,183
4,black,latinx,17


In [50]:
df_temp = df_temp.query("race in ('white', 'black', 'latinx')")   # only looking at the main 3

In [51]:
df_temp

,race,victim(s)_race(s),total
2,black,asian,12
3,black,black,183
4,black,latinx,17
5,black,multiple,15
6,black,other,3
7,black,white,302
8,latinx,asian,2
9,latinx,black,3
10,latinx,latinx,63
11,latinx,multiple,6


In [52]:
df_temp.sort_values(by=["race", "total"], ascending=False)

,race,victim(s)_race(s),total
24,white,white,798
22,white,multiple,23
20,white,black,21
21,white,latinx,20
19,white,asian,5
23,white,native american,2
10,latinx,latinx,63
12,latinx,white,56
11,latinx,multiple,6
9,latinx,black,3


In [53]:
df_temp_sum = ( df_temp.groupby(["race"])[["total"]]
                       .sum()
                       .reset_index()
              )

df_temp_sum

,race,total
0,black,532
1,latinx,130
2,white,869


In [54]:
df_temp = ( df_temp.merge(df_temp_sum, on=["race"], how="left")
                   .rename(columns={"total_x": "total_indiv", "total_y": "total_race"})
          )

df_temp.head()

,race,victim(s)_race(s),total_indiv,total_race
0,black,asian,12,532
1,black,black,183,532
2,black,latinx,17,532
3,black,multiple,15,532
4,black,other,3,532


In [55]:
df_temp["ratio_pct"] = df_temp.total_indiv / df_temp.total_race * 100

df_temp

,race,victim(s)_race(s),total_indiv,total_race,ratio_pct
0,black,asian,12,532,2.255639
1,black,black,183,532,34.398496
2,black,latinx,17,532,3.195489
3,black,multiple,15,532,2.819549
4,black,other,3,532,0.563910
5,black,white,302,532,56.766917
6,latinx,asian,2,130,1.538462
7,latinx,black,3,130,2.307692
8,latinx,latinx,63,130,48.461538
9,latinx,multiple,6,130,4.615385


In [56]:
cut_off_pct = 10

df_temp_other = df_temp.query("ratio_pct < @cut_off_pct").groupby(["race"])[["total_indiv", "ratio_pct"]].sum().reset_index()

df_temp_other["victim(s)_race(s)"] = "other"

df_temp_other

,race,total_indiv,ratio_pct,victim(s)_race(s)
0,black,47,8.834586,other
1,latinx,11,8.461538,other
2,white,71,8.170311,other


In [57]:
df_temp = ( pd.concat([df_temp.query("ratio_pct >= @cut_off_pct"), df_temp_other], axis=0)
              .sort_values(["race", "ratio_pct"], ascending=False)
              .drop(columns=["total_race"])
              .reset_index(drop=True)
          )

df_temp["race"] = df_temp["race"].str.title()
df_temp["victim(s)_race(s)"] = df_temp["victim(s)_race(s)"].str.title()

df_temp

,race,victim(s)_race(s),total_indiv,ratio_pct
0,White,White,798,91.829689
1,White,Other,71,8.170311
2,Latinx,Latinx,63,48.461538
3,Latinx,White,56,43.076923
4,Latinx,Other,11,8.461538
5,Black,White,302,56.766917
6,Black,Black,183,34.398496
7,Black,Other,47,8.834586


In [58]:
fig = px.sunburst(df_temp, path=["race", "victim(s)_race(s)"], values='total_indiv', 
                  title="Race(s) of perpetrator (inner section) and victim(s) (outer ring)")
fig.show()

Executed Whites overwhelmingly murdered other Whites. The victims of executed Blacks (or Latinx) were mainly composed of Whites and Blacks (or Latinx) in comparable proportions.  

# Summary

- Overall trend:
    - Executions clearly ramped up since their reinstatement in 1976, **peaked around the turn of the millenium**, and then steadily declined again thereafter.


- Characteristics of the executed individual: 
    - The executed were almost exclusively **male**.
    - The executed were almost exclusively **not juvenile when commiting the murder(s)**.
    - **Whites** and **Blacks** constitute the clear majority of the executed.
    - The executed were almost exclusively **not foreign nationals**.


- Location:
    - The vast majority of executions were carried out in the **South**.
    - A substantial part of total number of executions were carried out in **Texas**.


- Details of the murder:
    - In the vast majority of cases, there is a **single murder victim**. Higher victim numbers exist, however, at much lower frequencies.


- Voluntary execution:
    - Almost **10%** of executions involved a "**voluntary execution**"!
    - The vast majority of execution volunteers were **White**.
    
    
- Inter-racial aspects:
    - In **two thirds** of all cases the races of perpetrator and victim(s) **coincide**.
    - Executed Whites overwhelmingly murdered other Whites. The victims of executed Blacks (or Latinx) were mainly composed of Whites and Blacks (or Latinx) in comparable proportions.


**IMPORTANT CAVEATS**
- When reviewing the results of above analysis, it is at times rather easy to fall into the trap of confusing the trends isolated from the execution data with the ones for a "general crime statistic". It should thus be stressed that **executed individuals** are only a subset of **people who committed aggrevated murder, got caught, and received the death sentence**, which are only a subset of **people who committed aggrevated murder and got caught**, which are only a subset of **people who committed aggrevated murder**.
- Certain aspects of this analysis would benefit from taking into account further considerations, for example **normalising** the number of executions by the population of a given US state.